In [4]:
import os
from users.base_user import UserSim
from users.llm_user import LLMUserSim
from dialogue_sim import DialogueSimulator
import pe_modules
import users
import dataloaders
import llms
import jinja2
import argparse
import yaml
import json

In [15]:
#Read data

###Shapes:
All combinations of Color

In [19]:
#shapes
with open('data/shapes.json') as f:
    shapes = json.load(f)  

In [20]:
shapes

{'0': {'name': '0', 'description': 'Colour: Red, Size: Large, Shape: Circle'},
 '1': {'name': '1',
  'description': 'Colour: Red, Size: Large, Shape: Triangle'},
 '2': {'name': '2', 'description': 'Colour: Red, Size: Small, Shape: Circle'},
 '3': {'name': '3',
  'description': 'Colour: Red, Size: Small, Shape: Triangle'},
 '4': {'name': '4', 'description': 'Colour: Blue, Size: Large, Shape: Circle'},
 '5': {'name': '5',
  'description': 'Colour: Blue, Size: Large, Shape: Triangle'},
 '6': {'name': '6',
  'description': 'Colour: Blue, Size: Small, Shape: Triangle'},
 '7': {'name': '7',
  'description': 'Colour: Blue, Size: Small, Shape: Triangle'}}